In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('./../../data_processed/clean_data_small_v2.csv') 

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head()

,id,date_creat,nature,groupement,titre,titre_court,objet,objet_social1,adrg_declarant,adrg_complemid,adrg_complemgeo,adrg_libvoie,adrg_distrib,adrg_codepostal,adrg_achemine,adrg_pays,objet_social1_desc,object_social_group
0,W012000002,1983-10-10,D,S,ARIADNE,ARIADNE,Favoriser le développement d'une compagnie thé...,6070,NaN,NaN,NaN,39 rue Courteline,NaN,69100,villeurbanne,FRANCE,"théâtre, marionnettes, cirque, spectacles de v...",culture et loisirs
1,W012000005,2003-09-01,D,S,UNITE FRANCAISE D'INTERVENTION EN CATASTROPHE ...,UNITE FRANCAISE D'INTERVENTION EN ...,Intervenir lors de catastrophes naturelles sur...,20000,NaN,NaN,NaN,18 rue Ampère,NaN,69270,FONTAINE SUR SAONE,FRANCE,"associations caritatives, humanitaires, aide a...",action sociale
2,W033002617,2008-05-19,D,S,ASSOCIATION MAYOTTE FOOTBALL CLUB DE BELLERIVE...,MAYOTTE FOOTBALL CLUB BSA,pratiquer et développer le football; organisat...,11075,NaN,NaN,NaN,8 RUE EMMANUEL CHABRIER,NaN,03700,BELLERIVE-SUR-ALLIER,FRANCE,"Football (football, futsal)",sports
3,W423003687,2008-07-02,D,S,SOUL ALMIGHTY,SOUL ALMIGHTY,promouvoir la musique reggae tout en participa...,6030,NaN,NaN,NaN,36 rue de la Roche du Geai,NaN,42000,Saint-Étienne,FRANCE,"chant choral, musique",culture et loisirs
4,W441001349,2012-03-29,D,S,TUMBA MUSIC,TUMBA MUSIC,développer des actions culturelles de soutien ...,6030,NaN,NaN,NaN,6 rue du Boispéan,NaN,44110,Châteaubriant,FRANCE,"chant choral, musique",culture et loisirs


In [4]:
df.adrg_codepostal

0          69100
1          69270
2          03700
3          42000
4          44110
           ...  
1531240    95220
1531241    75020
1531242    02300
1531243    34560
1531244    95110
Name: adrg_codepostal, Length: 1531245, dtype: object

In [5]:
df.shape

(1531245, 18)

In [6]:
# mising postacodes - not many
df['adrg_codepostal'].isnull().sum()/1531245

0.0012107794637696775

In [7]:
from collections import Counter
post_codes = df.adrg_codepostal
strcount = 0
strings = []
fine = 0
count = Counter()

for index, row in post_codes.iteritems():
    try:
        tmp = str(int(float(row)))
        if len(tmp) == 5:
            fine += 1
            count[tmp] += 1  # Number of charities per postcode
        else:
            strcount += 1
    except ValueError:
        strcount += 1
        strings.append(row)

print(f'Fine: {fine}, Errors: {strcount}, NaNs (among errors): {post_codes.isna().sum()}')

Fine: 1441429, Errors: 89816, NaNs (among errors): 1854


From visual inspection possible formats include:
* 2640 str? int? (one i looked at was a float)
* 44100.0 float
* 4100.0 float
* 

converting to numerics and then as floats

In [8]:
#### setup clean column
df['adrg_codepostal_clean'] = np.nan

# find only rows with numerics in the dataframe column  
only_nums = df[pd.to_numeric(df.adrg_codepostal, errors='coerce').notnull()]
# convert the values in this series to strings
code_postal_ints = only_nums.adrg_codepostal.astype(float).astype(int).astype(str)
# add extra 0 to front of postcodes that should start with 0 that are missing
code_postal_ints = code_postal_ints.apply(lambda x : ('0' + x) if len(x) == 4 else x)
# assign clean strings series to adrg_codepostal_clean column 
df.loc[code_postal_ints.index, 'adrg_codepostal_clean'] = code_postal_ints.values 

df.adrg_codepostal_clean#.head()

0          69100
1          69270
2          03700
3          42000
4          44110
           ...  
1531240    95220
1531241    75020
1531242    02300
1531243    34560
1531244    95110
Name: adrg_codepostal_clean, Length: 1531245, dtype: object

Check clean postcode data column

In [59]:
df.adrg_codepostal_clean.isnull().sum()

1907

As well as the 1854 missed through being nans.

We only miss 53 more when we do the type conversion above, they look like this:

AD503
7198       B1410
9185       B3090
24049      8412O
39626      AAAAA
60135      E146P
60681      B4030
101995     3700o
273950     XXXXX
282451     L4962
298610     288fu
333305     FL331
376822     10O00
406569     CV130
484784     1216-
523127     SW129
551598     8547O
558172     B1170
575376     Paris
578416     B7782

In [39]:
df[df.adrg_codepostal_clean == 'empties'].adrg_codepostal

1826       AD503
7198       B1410
9185       B3090
24049      8412O
39626      AAAAA
60135      E146P
60681      B4030
101995     3700o
273950     XXXXX
282451     L4962
298610     288fu
333305     FL331
376822     10O00
406569     CV130
484784     1216-
523127     SW129
551598     8547O
558172     B1170
575376     Paris
578416     B7782
654723     L4499
673021     9/880
676054     7501ç
757593     B6790
763981     BR76D
788016     850G0
801596     561OO
821180     A1090
835713     122-8
850772     1017H
851845     XXXXX
869495     N11JJ
882200     CONGO
898311     IG80D
965220     .....
991182     W1A6U
1007327    31 42
1013504    8430O
1017260    B1340
1018511    1055S
1058513    1150-
1089325    0510O
1115739    B8940
1206460     BP22
1234545    B7010
1249284    3100O
1261391       BP
1286219    192O2
1367078    PARIS
1476271    CC700
1479960    5935O
1506370    B1040
1521221    83O56
Name: adrg_codepostal, dtype: object

In [34]:
1900/df.shape[0]

0.00124082037818899

In [36]:
df.adrg_codepostal_clean.astype(str)

0          69100
1          69270
2           3700
3          42000
4          44110
           ...  
1531240    95220
1531241    75020
1531242     2300
1531243    34560
1531244    95110
Name: adrg_codepostal_clean, Length: 1531245, dtype: object

In [33]:
df

,id,date_creat,nature,groupement,titre,titre_court,objet,objet_social1,adrg_declarant,adrg_complemid,adrg_complemgeo,adrg_libvoie,adrg_distrib,adrg_codepostal,adrg_achemine,adrg_pays,objet_social1_desc,object_social_group,adrg_codepostal_clean
0,W012000002,1983-10-10,D,S,ARIADNE,ARIADNE,Favoriser le développement d'une compagnie thé...,6070,NaN,NaN,NaN,39 rue Courteline,NaN,69100,villeurbanne,FRANCE,"théâtre, marionnettes, cirque, spectacles de v...",culture et loisirs,69100
1,W012000005,2003-09-01,D,S,UNITE FRANCAISE D'INTERVENTION EN CATASTROPHE ...,UNITE FRANCAISE D'INTERVENTION EN ...,Intervenir lors de catastrophes naturelles sur...,20000,NaN,NaN,NaN,18 rue Ampère,NaN,69270,FONTAINE SUR SAONE,FRANCE,"associations caritatives, humanitaires, aide a...",action sociale,69270
2,W033002617,2008-05-19,D,S,ASSOCIATION MAYOTTE FOOTBALL CLUB DE BELLERIVE...,MAYOTTE FOOTBALL CLUB BSA,pratiquer et développer le football; organisat...,11075,NaN,NaN,NaN,8 RUE EMMANUEL CHABRIER,NaN,03700,BELLERIVE-SUR-ALLIER,FRANCE,"Football (football, futsal)",sports,3700
3,W423003687,2008-07-02,D,S,SOUL ALMIGHTY,SOUL ALMIGHTY,promouvoir la musique reggae tout en participa...,6030,NaN,NaN,NaN,36 rue de la Roche du Geai,NaN,42000,Saint-Étienne,FRANCE,"chant choral, musique",culture et loisirs,42000
4,W441001349,2012-03-29,D,S,TUMBA MUSIC,TUMBA MUSIC,développer des actions culturelles de soutien ...,6030,NaN,NaN,NaN,6 rue du Boispéan,NaN,44110,Châteaubriant,FRANCE,"chant choral, musique",culture et loisirs,44110
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531240,W951006063,2003-05-12,D,S,SAVOIR ET CONNAISSANCE,SAVOIR ET CONNAISSANCE,combiner la culture et l'ouverture sur le mond...,15000,NaN,NaN,NaN,6 RUE PIERRE LOTI,NaN,95220,HERBLAY,FRANCE,éducation formation,éducation,95220
1531241,W751072587,1982-05-17,D,S,ASSOCIATION SPORTIVE CAMBODGIENNE AS CAMBODGIENNE,ASSOCIATION SPORTIVE CAMBODGIENNE A...,"développer la pratique du sport loisir, princi...",11135,NaN,NaN,NaN,32 RUE HENRI CHEVREAU,NaN,75020,PARIS 20,FRANCE,"Sports aériens (avion, planeur, ULM, parachuti...",sports,75020
1531242,W022006227,1996-01-21,D,S,"MOUVEMENT FRANCAIS POUR LE PLANNING FAMILIAL, ...",PLANNING FAMILIAL 02,lutter pour le droit à l'information et a l'éd...,19000,NaN,PRéSIDENTE DE L'ASSOCIATION,NaN,74 BOULEVARD GAMBETTA,BP 80028,02300,CHAUNY,FRANCE,interventions sociales,action sociale,2300
1531243,W343026793,1934-12-18,D,S,ASSOCIATION D EDUCATION POPULAIRE SAINT VINCENT,ASSOCIATION D EDUCATION POPULAIRE S...,"gérer, entretenir et restaurer, au plus juste ...",20000,NaN,NaN,NaN,4 CHEMIN DES FRèRES,NaN,34560,POUSSAN,FRANCE,"associations caritatives, humanitaires, aide a...",action sociale,34560


In [51]:
1529338/1531245

0.9987546081783124

In [23]:
dftest = pd.DataFrame(index=['A', 'B', 'C'], columns=['C1', 'C2'])  # original df
s = pd.Series({'A': 'spam'})  # a computed result
index = pd.Index(['A', 'C'])

In [24]:
dftest.loc[index, 'C2'] = s.values

In [25]:
s

A    spam
dtype: object

In [26]:
dftest

,C1,C2
A,NaN,spam
B,NaN,NaN
C,NaN,spam


In [ ]:
non_nulls = df[df.adrg_codepostal.notnull()]['adrg_codepostal']
non_nulls.is

In [35]:
# post_codes = df['adrg_codepostal']

non_nulls = df[df.adrg_codepostal.notnull()]['adrg_codepostal']


def my_fun():
    for index, row in non_nulls[0:10].iteritems():
        try:
            tmp = str(int(row))
            if (len(tmp) == 5)|(len(tmp) == 4):
                df.loc[index,'adrg_codepostal_clean'] = row
            else:
                print(row)
        except ValueError:
            print(row)
            
%timeit my_fun()

# non_nulls.adrg_codepostal.astype(float)

499 ms ± 27.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
# post_codes = df['adrg_codepostal']

non_nulls = df[df.adrg_codepostal.notnull()]['adrg_codepostal']


def my_fun():
    for index, row in non_nulls[0:10].iteritems():
        try:
            tmp = str(int(row))
            if (len(tmp) == 5):
                df.loc[index,'adrg_codepostal_clean'] = row
        except ValueError:
            
            return
#             else:
#                 print(row)
#         except ValueError:
#             print(row)
            
%timeit my_fun()

# non_nulls.adrg_codepostal.astype(float)

438 ms ± 5.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
for index, row in post_codes.iteritems():
    try:
        tmp = str(int(row))
        if (len(tmp) == 5)|(len(tmp) == 4):
            df.loc[index,'adrg_codepostal_clean'] = row
        else:
            print(row)
    except ValueError:
        print(row)


non_nulls.adrg_codepostal.astype(float)

nan
nan
nan
AD503
nan
nan
nan
nan
nan
B1410
nan
nan
B3090
nan
nan
nan
nan
nan


KeyboardInterrupt: 

Checks for whats been transferred.

In [ ]:
df.adrg_codepostal_clean.isnull().sum()

In [ ]:
df[df.adrg_codepostal_clean == 'empty'].shape

Faster implementation

In [ ]:
non_nulls = df[df.adrg_codepostal.notnull()]['adrg_codepostal']

In [ ]:
df['adrg_codepostal_clean']

In [9]:
df.adrg_codepostal.astype() #str(int(float(row)))

0          69100
1          69270
2          03700
3          42000
4          44110
           ...  
1531240    95220
1531241    75020
1531242    02300
1531243    34560
1531244    95110
Name: adrg_codepostal, Length: 1531245, dtype: object

In [ ]:
for i in df.adrg_codepostal[100000:]:
    if (type(i) != str):#|(len(i) != 5):
        print(i)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
75005.0
30700.0
15500.0
40260.0
94320.0
49510.0
47190.0
35500.0
47470.0
62113.0
97435.0
64460.0
97430.0
47000.0
47310.0
47000.0
47000.0
10600.0
47000.0
47000.0
24260.0
44000.0
44000.0
21000.0
11170.0
47130.0
13013.0
64530.0
92250.0
9700.0
9140.0
31400.0
59552.0
83200.0
92200.0
25000.0
47000.0
97200.0
47000.0
91150.0
35000.0
97430.0
47360.0
47470.0
nan
47000.0
47000.0
73350.0
47000.0
31110.0
87260.0
55260.0
71150.0
47000.0
16100.0
87000.0
42370.0
47000.0
47130.0
62122.0
63170.0
75009.0
44410.0
59700.0
21270.0
34120.0
47520.0
60660.0
59140.0
95700.0
33800.0
47480.0
70100.0
12200.0
47240.0
2100.0
35480.0
3370.0
45120.0
5500.0
13140.0
47000.0
42840.0
35703.0
12500.0
6000.0
35600.0
97400.0
47000.0
33300.0
92310.0
35330.0
52400.0
56410.0
80000.0
92100.0
15000.0
94700.0
82000.0
91130.0
63730.0
47000.0
59420.0
47470.0
28700.0
33160.0
98800.0
49510.0
75011.0
47130.0
12

79700.0
37170.0
12260.0
97410.0
29201.0
26600.0
69130.0
50500.0
62290.0
64510.0
97434.0
49510.0
79250.0
50620.0
86000.0
59100.0
97214.0
13820.0
33000.0
79250.0
22540.0
31500.0
55190.0
25660.0
93130.0
13470.0
12100.0
17440.0
16130.0
73170.0
79290.0
44600.0
79250.0
85300.0
98820.0
49400.0
45299.0
5400.0
29490.0
52330.0
13090.0
98810.0
85250.0
59195.0
98829.0
15700.0
98880.0
79140.0
79300.0
20240.0
97360.0
71150.0
13007.0
60560.0
10000.0
35235.0
11100.0
33100.0
94170.0
95430.0
94270.0
95300.0
80700.0
79140.0
93500.0
94400.0
72440.0
94210.0
77590.0
40300.0
69380.0
93100.0
94120.0
75020.0
94510.0
64160.0
52000.0
52700.0
52000.0
79140.0
13700.0
93390.0
4190.0
43270.0
52000.0
45160.0
32140.0
52120.0
54000.0
93800.0
30240.0
52000.0
63610.0
52210.0
12200.0
79350.0
93220.0
52320.0
52000.0
60130.0
79140.0
88200.0
85140.0
93170.0
84000.0
3140.0
69003.0
69001.0
12300.0
63000.0
93500.0
11190.0
97224.0
52200.0
93800.0
77760.0
93120.0
94700.0
17350.0
33210.0
21290.0
56320.0
44380.0
27340.0
93100.0
242

84210.0
64000.0
13009.0
19000.0
18570.0
44190.0
18200.0
42800.0
82340.0
13600.0
18100.0
91160.0
9230.0
17380.0
24600.0
78640.0
77100.0
75015.0
38000.0
31800.0
56000.0
65700.0
52100.0
1500.0
85000.0
46160.0
38300.0
59262.0
52100.0
52300.0
80210.0
62970.0
76230.0
97200.0
9100.0
52100.0
17770.0
38120.0
81150.0
52300.0
52300.0
3100.0
85500.0
31400.0
30840.0
97200.0
1800.0
35240.0
86000.0
59000.0
93170.0
59320.0
17130.0
36000.0
52220.0
59650.0
52300.0
56100.0
13540.0
56170.0
31000.0
64400.0
59870.0
20230.0
53800.0
33510.0
17430.0
31000.0
34970.0
59510.0
72650.0
60150.0
76700.0
64000.0
77760.0
14380.0
45190.0
35300.0
5230.0
15700.0
33700.0
91150.0
31450.0
55200.0
9290.0
46350.0
13015.0
29550.0
65000.0
87800.0
14420.0
15130.0
75012.0
97419.0
13008.0
59200.0
6000.0
69005.0
89000.0
14500.0
30160.0
29200.0
53000.0
42100.0
12400.0
92120.0
53150.0
14320.0
11130.0
35230.0
21000.0
80450.0
62790.0
69960.0
62750.0
53600.0
64510.0
53600.0
89190.0
59128.0
29400.0
14440.0
13530.0
13610.0
42100.0
8400.0
7

84200.0
89100.0
34160.0
45100.0
22320.0
80000.0
51120.0
92000.0
92400.0
17600.0
17460.0
79330.0
54630.0
24150.0
50140.0
17000.0
63000.0
37200.0
17000.0
17250.0
13001.0
19160.0
36350.0
55300.0
9000.0
17200.0
17450.0
97490.0
89300.0
64300.0
47600.0
54500.0
56130.0
55130.0
54000.0
59603.0
97200.0
19100.0
50200.0
54180.0
16290.0
16160.0
77720.0
16100.0
76240.0
80240.0
46400.0
15100.0
97438.0
62127.0
92290.0
16000.0
13003.0
62218.0
13100.0
14500.0
38160.0
71390.0
28480.0
45140.0
54320.0
59500.0
30500.0
34200.0
12250.0
69420.0
59370.0
37800.0
40990.0
53500.0
14100.0
24660.0
66200.0
85400.0
54520.0
54000.0
13100.0
54700.0
34520.0
75003.0
13009.0
64500.0
44150.0
13100.0
44850.0
13002.0
13090.0
56360.0
38000.0
34160.0
76660.0
1600.0
44330.0
82110.0
6200.0
60200.0
98830.0
13012.0
13012.0
16000.0
92110.0
38300.0
33000.0
13127.0
70130.0
28500.0
65390.0
83200.0
24430.0
98830.0
32380.0
59000.0
13015.0
13003.0
62880.0
97427.0
14200.0
69364.0
87350.0
25200.0
49300.0
1400.0
89150.0
64530.0
58000.0
9720

54770.0
29160.0
59430.0
54000.0
17390.0
29000.0
74450.0
30900.0
2640.0
38650.0
54500.0
54000.0
47000.0
92340.0
21240.0
54000.0
28170.0
29190.0
59120.0
26400.0
31100.0
59273.0
82000.0
75016.0
87230.0
57570.0
92200.0
54630.0
54710.0
31400.0
37540.0
92130.0
78320.0
75009.0
10000.0
54000.0
34400.0
54000.0
4120.0
34250.0
39190.0
54390.0
28140.0
54000.0
38690.0
97310.0
60000.0
44680.0
75019.0
78500.0
70240.0
24170.0
90800.0
54520.0
54000.0
94100.0
33230.0
97200.0
1124.0
59190.0
69100.0
54320.0
54112.0
35000.0
54500.0
88450.0
38270.0
54000.0
54000.0
26310.0
54520.0
59350.0
7800.0
54000.0
26150.0
75018.0
29270.0
21200.0
nan
62190.0
97200.0
80690.0
54390.0
18000.0
24660.0
29690.0
63360.0
59115.0
71600.0
4000.0
80310.0
54710.0
54500.0
54500.0
38390.0
7000.0
38110.0
24750.0
54130.0
54000.0
56930.0
21460.0
54510.0
54380.0
93200.0
54500.0
33000.0
97424.0
34400.0
55190.0
54000.0
69004.0
83500.0
28210.0
66140.0
62290.0
64570.0
54500.0
54000.0
59800.0
70000.0
33220.0
77200.0
54000.0
44310.0
54602.0
34

...

86220.0
75012.0
81600.0
97123.0
55100.0
98600.0
63160.0
35160.0
75004.0
14000.0
80000.0
87000.0
6130.0
44400.0
75015.0
75003.0
29750.0
78120.0
49000.0
1360.0
6230.0
71100.0
15000.0
27620.0
70190.0
78280.0
77700.0
4100.0
56890.0
78110.0
22590.0
56650.0
69220.0
97400.0
86200.0
5350.0
72000.0
30140.0
77400.0
97354.0
79300.0
94450.0
33750.0
14840.0
93400.0
69360.0
85140.0
74400.0
92400.0
93200.0
93400.0
20146.0
69480.0
21000.0
75012.0
13127.0
59980.0
63970.0
13290.0
95110.0
95100.0
64110.0
50450.0
2210.0
92100.0
6000.0
33270.0
97410.0
75011.0
25000.0
93140.0
69640.0
98718.0
75015.0
24620.0
24120.0
4300.0
17400.0
97232.0
25560.0
33000.0
80400.0
69480.0
98809.0
29120.0
11440.0
95100.0
97122.0
98800.0
13300.0
64140.0
33200.0
95800.0
69550.0
97440.0
38800.0
38220.0
98800.0
33150.0
95220.0
98800.0
95880.0
94190.0
76500.0
38560.0
92140.0
35000.0
75007.0
44322.0
82100.0
80100.0
34820.0
59610.0
17580.0
42600.0
63110.0
69620.0
11200.0
75011.0
69620.0
42700.0
23210.0
84170.0
73000.0
32003.0
30220.0


97420.0
47120.0
87000.0
76000.0
81100.0
25150.0
42400.0
27680.0
7800.0
82100.0
72100.0
12450.0
38180.0
28210.0
40180.0
76220.0
87000.0
21000.0
38000.0
59144.0
78500.0
54290.0
26410.0
59640.0
91550.0
51100.0
33670.0
30800.0
62145.0
8140.0
17940.0
17220.0
31200.0
13320.0
64200.0
76540.0
56340.0
93100.0
64170.0
36360.0
31620.0
44270.0
13821.0
56400.0
78520.0
69675.0
13120.0
64400.0
69100.0
97410.0
35000.0
91170.0
44270.0
22310.0
91630.0
25000.0
83143.0
84240.0
95470.0
16000.0
39570.0
64120.0
53600.0
10300.0
97150.0
66600.0
50270.0
75008.0
76000.0
36400.0
55290.0
50200.0
88380.0
75012.0
78125.0
33360.0
47170.0
97351.0
7110.0
59000.0
84200.0
78740.0
82100.0
14340.0
30440.0
41250.0
74600.0
54500.0
47200.0
12500.0
74100.0
13100.0
12800.0
13140.0
75012.0
91420.0
94120.0
88120.0
38480.0
75005.0
42450.0
35160.0
36000.0
13460.0
92400.0
84000.0
63210.0
94380.0
25000.0
97232.0
62470.0
77500.0
59580.0
59700.0
49400.0
84000.0
91220.0
80000.0
28000.0
80340.0
7170.0
97118.0
34000.0
82330.0
29370.0
7200

19350.0
80300.0
43170.0
93200.0
7100.0
69430.0
53000.0
33110.0
34700.0
85500.0
34270.0
3200.0
35850.0
75015.0
94450.0
23800.0
11130.0
56260.0
83230.0
29790.0
12270.0
29540.0
75006.0
95400.0
31520.0
58000.0
89005.0
83200.0
53270.0
6000.0
42580.0
81000.0
73100.0
34070.0
55190.0
55000.0
74460.0
13002.0
35000.0
97441.0
25660.0
1960.0
94220.0
47320.0
79000.0
51100.0
24350.0
1390.0
86240.0
26400.0
20124.0
24330.0
51170.0
77700.0
34370.0
54500.0
75017.0
95500.0
51220.0
18200.0
12380.0
86340.0
87110.0
84210.0
38510.0
91100.0
84220.0
37550.0
92600.0
2550.0
93500.0
65000.0
97200.0
78300.0
56340.0
87000.0
77130.0
75008.0
25000.0
50860.0
47440.0
56420.0
47300.0
86000.0
16360.0
87100.0
62000.0
33600.0
87220.0
2290.0
58110.0
69005.0
13300.0
90300.0
80000.0
76480.0
95750.0
87000.0
34000.0
25600.0
75018.0
63490.0
31100.0
59990.0
69210.0
11110.0
91090.0
59910.0
56950.0
13800.0
21000.0
97419.0
63000.0
7170.0
81600.0
30900.0
22950.0
31320.0
95300.0
2800.0
97129.0
3360.0
95550.0
89400.0
31000.0
83210.0
29

41200.0
63200.0
36230.0
97490.0
80300.0
nan
50110.0
43700.0
13124.0
94200.0
44100.0
31350.0
59630.0
33170.0
49041.0
60180.0
75020.0
60160.0
91000.0
13127.0
76620.0
95300.0
71880.0
88500.0
93270.0
88250.0
2250.0
97318.0
84100.0
2700.0
75016.0
63290.0
95190.0
13340.0
97320.0
59287.0
81000.0
22120.0
25640.0
59653.0
31340.0
58320.0
97170.0
75013.0
38200.0
25700.0
83170.0
87000.0
76220.0
97490.0
74500.0
77720.0
15600.0
50000.0
34730.0
70150.0
60290.0
84140.0
17139.0
97600.0
33370.0
69001.0
40090.0
32170.0
36000.0
34400.0
74200.0
76260.0
49510.0
34620.0
42560.0
60160.0
75015.0
44420.0
56100.0
39000.0
1160.0
1120.0
30220.0
75020.0
30760.0
8120.0
35560.0
18320.0
25000.0
97400.0
59113.0
54710.0
28630.0
83860.0
48500.0
77270.0
33000.0
18160.0
21121.0
2000.0
33000.0
71960.0
24440.0
69620.0
25370.0
75014.0
50800.0
51100.0
2550.0
59640.0
6300.0
92500.0
54000.0
30300.0
87000.0
44550.0
77122.0
75009.0
31880.0
59470.0
80520.0
14640.0
41300.0
29100.0
59110.0
97441.0
72610.0
38090.0
16230.0
60880.0
1817

93370.0
69003.0
6210.0
95800.0
6230.0
7000.0
29460.0
59230.0
75009.0
56100.0
75020.0
21000.0
75020.0
72320.0
33130.0
78500.0
62127.0
14000.0
7700.0
75003.0
80200.0
64600.0
48200.0
13004.0
71700.0
97300.0
83310.0
18150.0
91210.0
18600.0
94190.0
83560.0
78120.0
71140.0
51160.0
1370.0
43800.0
74130.0
75005.0
11230.0
92360.0
87220.0
75003.0
17190.0
72000.0
35190.0
55100.0
37510.0
13150.0
74260.0
49122.0
28260.0
60300.0
44110.0
72200.0
69130.0
95140.0
92160.0
78830.0
30570.0
44000.0
87340.0
97110.0
31170.0
97490.0
69530.0
62187.0
64000.0
15340.0
59280.0
6110.0
66400.0
13004.0
88530.0
93100.0
72000.0
34300.0
18000.0
80000.0
59300.0
77120.0
38550.0
76240.0
97670.0
42800.0
76500.0
58240.0
13014.0
16370.0
29810.0
65300.0
73500.0
91550.0
97320.0
33250.0
75011.0
69440.0
75007.0
80800.0
69480.0
56590.0
7000.0
59000.0
56400.0
70400.0
70300.0
77430.0
28630.0
34000.0
77380.0
12450.0
62200.0
97232.0
92120.0
80100.0
69600.0
75009.0
13013.0
97487.0
29700.0
59350.0
45550.0
7300.0
69004.0
83190.0
7190.0
6

In [34]:
#### transfer nans
df.loc[df[df['adrg_codepostal'].isnull()].index, 'adrg_codepostal_clean'] = np.nan

In [39]:
#### transfer tidy postcodes (either int already or convert floats )
tidy_ = df.adrg_codepostal.astype(int, errors='ignore').astype(str)

0          69100
1          69270
2          03700
3          42000
4          44110
           ...  
1531240    95220
1531241    75020
1531242    02300
1531243    34560
1531244    95110
Name: adrg_codepostal, Length: 1531245, dtype: object

In [80]:
df.adrg_codepostal_clean.isnull().sum()

1854

In [77]:
#### transfer clean postcodes


In [26]:
df.adrg_codepostal[0]

'69100'

In [30]:
df['adrg_codepostal_clean'] = 'empty'

for index, row in df.adrg_codepostal[0:10000].iteritems():
    
    if (type(row) == np.nan): # transfer nans
        df.loc[index,'adrg_codepostal_clean'] = np.nan
    
    elif (type(row) == str) & (len(row) == 5): # transfer clean postcodes
        df.loc[index,'adrg_codepostal_clean'] = row
    
    elif (type(row) == str) & (len(row) != 5):
        print(row)
        
        

TypeError: object of type 'float' has no len()

In [21]:
df.adrg_codepostal_clean.value_counts()

empty    1529391
Name: adrg_codepostal_clean, dtype: int64

In [22]:
df.adrg_codepostal_clean.isnull().sum()

1854

In [62]:
type(df[df.adrg_codepostal == 2640].adrg_codepostal[88273])

float

#### Function to clean

In [34]:
df['code_postal'] = df.adrg_codepostal.astype(str)

In [27]:
df.adrg_codepostal[0:10].astype(str)

0    69100
1    69270
2    03700
3    42000
4    44110
5    98728
6    88200
7    42660
8    98822
9    44320
Name: adrg_codepostal, dtype: object